# Problem 3
https://projecteuler.net/problem=3

In [1]:
println("Julia v", VERSION)

Julia v1.11.1


In [2]:
using BenchmarkTools

### Solution 1 (Sieve of Eratosthenes, generator)

In [3]:
function eratosthenes(c::Channel)
    D = Dict{Int64, Vector{Int64}}()
    q = 2
    while true
        if !(q in keys(D))
            D[q * q] = Int64[q]
            put!(c, q)
        else
            for p in D[q]
                haskey(D, p+q) ? push!(D[p+q], p) : D[p+q] = [p]
            end
            delete!(D, q)
        end
        q += 1
    end
end

sieve = Channel(eratosthenes)

@assert last([take!(sieve) for _ in 1:10]) == 29 "10th prime should be 29"

In [4]:
function max_prime_factor(n)
    sieve = Channel(eratosthenes)
    prime_factors = Int64[]
    while true
        p = take!(sieve)
        if n == p
            push!(prime_factors, p)
            return p
        end
        while n%p == 0
            push!(prime_factors, p)
            n ÷= p
        end
    end
end

@assert max_prime_factor(13195) == 29

In [5]:
@btime max_prime_factor(600851475143)

  22.230 ms (18483 allocations: 833.08 KiB)


6857

### Solution 2 (Primes package)

In [6]:
using Primes

In [7]:
@btime maximum(keys(Primes.factor(600851475143)))

  1.704 μs (2 allocations: 128 bytes)


6857

### Save benchmark

In [8]:
include("./euler.jl");
bm = @benchmark max_prime_factor(600851475143)
euler.save_benchmark(3, bm);